In [ ]:
%matplotlib inline


# Классификация видов комаров

Это руководство демонстрирует, как использовать `MosquitoClassifier` из библиотеки CulicidaeLab
для определения видов комаров по изображениям. Мы пройдем через весь процесс,
от загрузки модели до оценки ее производительности на пакете данных.

Это руководство охватывает:

- **Инициализация**: Как загрузить настройки и предварительно обученную модель.
- **Обработка данных**: Как использовать `DatasetsManager` для получения образцов данных.
- **Предсказание для одного изображения**: Как классифицировать одно изображение комара.
- **Визуализация**: Как интерпретировать и визуализировать предсказания модели.
- **Пакетная оценка**: Как измерить точность модели на наборе тестовых изображений.
- **Отчетность**: Как сгенерировать и визуализировать всесторонний отчет о производительности.


Установите библиотеку `culicidaelab`, если она еще не установлена
```bash
!pip install -q culicidaelab[full]
```
или, если есть доступ к GPU
```bash
!pip install -q culicidaelab[full-gpu]
```


## 1. Инициализация и настройка

Наш первый шаг — настройка необходимых компонентов. Мы инициализируем:

- **`settings`**: Объект, который содержит всю конфигурацию библиотеки, такую как
  пути к моделям и другие настройки.
- **`DatasetsManager`**: Класс для загрузки и управления образцами
  наборов данных, используемых в этом руководстве.
- **`MosquitoClassifier`**: Основной класс для выполнения задачи классификации. Мы передадим
  `load_model=True`, чтобы убедиться, что веса предварительно обученной модели будут загружены
  в память сразу же.



Импорт необходимых библиотек для визуализации



In [ ]:
import matplotlib.pyplot as plt

# Импорт требуемых классов из библиотеки CulicidaeLab
from culicidaelab import (
    DatasetsManager,
    MosquitoClassifier,
    get_settings,
)

# Получаем экземпляр настроек библиотеки по умолчанию
settings = get_settings()

# Инициализируем сервисный класс, необходимый для управления и загрузки данных

manager = DatasetsManager(settings)

# Создаем экземпляр классификатора и загружаем модель.
# При первом запуске это может занять некоторое время, так как будут загружаться веса модели.
print("Инициализация MosquitoClassifier и загрузка модели...")
classifier = MosquitoClassifier(settings, load_model=True)
print("Модель успешно загружена.")

### Просмотр классов модели

Прежде чем начать определение видов, полезно знать, какие виды модель была
обучена распознавать. Мы можем легко получить доступ к этой информации из объекта
настроек.



In [ ]:
species_map = settings.species_config.species_map
print(f"--- Модель может распознавать {len(species_map)} классов ---")
# Выведем первые 5 для краткости
for idx, name in list(species_map.items())[:5]:
    print(f"  Индекс класса {idx}: {name}")
print("  ...")

## 2. Загрузка тестового набора данных

В этом руководстве мы будем использовать встроенный тестовый набор данных, предоставляемый библиотекой.
`DatasetsManager` упрощает загрузку и использование этих данных. Набор данных
содержит изображения и соответствующие им правильные метки классов (виды комаров),
которые мы будем использовать для сравнения с результатами работы классификатора
и их последующей оценки.



In [ ]:
print("\n--- Загрузка тестовой части ('test' split) набора данных 'classification' ---")
classification_test_data = manager.load_dataset("classification", split="test")
print("Тестовый набор данных успешно загружен!")
print(f"Количество образцов в тестовом наборе данных: {len(classification_test_data)}")

# Давайте выберем один образец для работы.
# Образец представляет собой словарь, содержащий изображение и его истинную метку.
classification_test_data = classification_test_data.shuffle(seed=35)
sample_index = 285
sample = classification_test_data[sample_index]
image = sample["image"]
ground_truth_label = sample["label"]

print(f"\nИстинная метка выбранного образца: '{ground_truth_label}'")

# Отображаем входное изображение
plt.figure(figsize=(6, 6))
plt.imshow(image)
plt.title(f"Входное изображение\n(Истинная метка: {ground_truth_label})")
plt.axis("off")
plt.show()

## 3. Классификация одного изображения

Теперь мы будем использовать классификатор для определения вида комара на нашем
выбранном изображении. Метод `predict()` принимает изображение (в виде массива NumPy,
пути к файлу или объекта PIL Image) и возвращает список видов комаров, отсортированный
от наиболее до наименее вероятного.



Запускаем классификацию на нашем образце изображения



In [ ]:
result = classifier.predict(image)

# Выводим топ-5 предсказаний в читаемом формате
print("--- Топ-5 предсказаний ---")
for p in result.predictions[:5]:
    print(f"{p.species_name}: {p.confidence:.2%}")

## 4. Визуализация и интерпретация результатов

Необработанный список определённых классификатором видов полезен,
но визуализации делают результаты гораздо более понятными. Мы создадим два графика:

1.  **Гистограмма**: Показывает уверенность модели для каждого возможного
    вида. Это отлично подходит не только для просмотра лучшего варианта ответа, но и для
    того, чтобы увидеть, какие другие виды и их вероятности определила модель.
2.  **Подписанное изображение**: Использует встроенный метод `visualize()` для создания
    итогового изображения, которое отображает топ-5 вариантов, определённых моделью, рядом с исходным изображением.



Создаем гистограмму для визуализации вероятностей для всех видов



In [ ]:
plt.figure(figsize=(10, 8))

# Предсказания уже отсортированы, поэтому мы можем их сразу построить
species_names = [p.species_name for p in result.predictions]
probabilities = [p.confidence for p in result.predictions]

# Мы развернем списки (`[::-1]`), чтобы самая высокая вероятность была вверху
bars = plt.barh(species_names[::-1], probabilities[::-1])

# Выделяем столбцы, которые соответствуют нашему порогу уверенности
conf_threshold = settings.get_config("predictors.classifier.confidence")
for bar in bars:
    if bar.get_width() >= conf_threshold:
        bar.set_color("teal")
    else:
        bar.set_color("lightgray")

# Добавляем контрольную линию для порога уверенности
plt.axvline(
    x=conf_threshold,
    color="red",
    linestyle="--",
    label=f"Порог уверенности ({conf_threshold:.0%})",
)
plt.xlabel("Присвоенная вероятность")
plt.title("Вероятности классификации видов")
plt.legend()
plt.tight_layout()
plt.show()

Теперь давайте используем встроенный визуализатор для получения итогового изображения



In [ ]:
annotated_image = classifier.visualize(image, result)

plt.figure(figsize=(10, 6))
plt.imshow(annotated_image)
plt.title("Результат классификации")
plt.axis("off")
plt.show()

## 5. Оценка точности модели с использованием пакета данных

Хотя классификация одного изображения полезна, более строгая проверка включает
оценку качества модели на всем наборе данных. Метод
`evaluate_batch()` как раз для этого предназначен. Он обрабатывает пакет
изображений и их соответствующие истинные метки, а затем вычисляет агрегированные
метрики.

Результатом является отчёт `report`, содержащий ключевые метрики, такие как средняя
точность и **матрицу ошибок**, которая точно показывает, где модель
преуспевает или ошибается.



Для примера оценим первые 30 изображений из тестового набора



In [ ]:
num_samples_to_evaluate = 30
batch_samples = classification_test_data.select(range(num_samples_to_evaluate))
batch_images = [sample["image"] for sample in batch_samples]
ground_truths = [sample["label"] for sample in batch_samples]

print(f"\n--- Оценка пакета из {len(batch_images)} изображений ---")

# Запускаем пакетную оценку.
# Метод может принимать результаты определения видов и истинные метки для их сравнения,
# или вызывать классификатор для получения результатов перед оценкой, если предоставлены
# только изображения.
report = classifier.evaluate_batch(
    input_data_batch=batch_images,
    ground_truth_batch=ground_truths,
    show_progress=True,
)

print("\n--- Сводка отчета об оценке ---")
for key, value in report.items():
    if key != "confusion_matrix":
        # Проводим форматирование в зависимости от типа значения
        if isinstance(value, float):
            print(f"  {key}: {value:.4f}")
        else:
            print(f"  {key}: {value}")

## 6. Визуализация отчета с оценкой качества модели
Сгенерированный словарь `report` содержит много информации, но
матрица ошибок лучше всего воспринимается визуально. Метод `visualize_report()`
создает комплексный график, который демонстрирует результаты оценки.

**Как читать матрицу ошибок:**
- Каждая строка представляет *фактический* (истинный) вид.
- Каждый столбец представляет вид, который *определила модель*.
- Диагональ (от верхнего левого до нижнего правого угла) показывает количество правильных
  ответов для каждого класса.
- Числа вне диагонали указывают на неверные классификации. Например, число
  в строке «A» и столбце «B» означает, что изображение вида A было неверно
  классифицировано как вид B.



Передаем отчет в функцию визуализации



In [ ]:
classifier.visualize_report(report)

## 7. Пакетное предсказание для обеспечения эффективных вычислений

Если цель — классифицировать много изображений, использование `predict_batch()`
гораздо эффективнее, чем итерация по `predict()`.
Этот метод может быть в разы эффективнее при использовании GPU для параллельной обработки изображений.



Мы будем использовать тот же небольшой пакет из нашего примера оценки



In [ ]:
print(
    f"\n--- Классификация пакета из {len(batch_images)} изображений с помощью predict_batch ---",
)
batch_predictions = classifier.predict_batch(batch_images, show_progress=True)

print("\n--- Результаты пакетной классификации (Лучшее предсказание для каждого изображения) ---")
for i, single_image_preds in enumerate(batch_predictions):
    if single_image_preds:  # Проверяем, что список с результатами не пуст
        top_pred_species = single_image_preds.predictions[0].species_name
        top_pred_conf = single_image_preds.predictions[0].confidence
        print(
            f"  - Изображение {i+1} (Истинная метка: {ground_truths[i]}): "
            f"Предсказан '{top_pred_species}' с уверенностью {top_pred_conf:.2%}.",
        )
    else:
        print(f"  - Изображение {i+1} (Истинная метка: {ground_truths[i]}): Предсказание не удалось.")